# Setup

Note that this notebook was developed with NodePy version 0.7.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk, stability_function

rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

short_methods = [ssp2, ssp3, rk4, bs5]
short_method_labels = ["SSPRK(2,2)", "SSPRK(3,3)", "RK(4,4)", "BSRK(8,5)"]

import os
if not os.path.isdir('./figures'):
    os.mkdir('./figures')

In [ ]:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 25}

import matplotlib
matplotlib.rc('font', **font)

In [ ]:
# line cycler adapted to colourblind people
from cycler import cycler
colors = ['#E69F00', '#56B4E9', '#009E73', '#0072B2', '#D55E00', '#CC79A7', '#F0E442']
linestyles = ['-', '--', '-.', ':', "-", "--", "-."]
markers = ['o','D','X','s','v']
colourblind_cycler = (cycler(color=colors) +
                      cycler(linestyle=linestyles))
plt.rc("axes", prop_cycle=colourblind_cycler)

plt.rc("text", usetex=True)
plt.rc("text.latex", 
       preamble=r"\usepackage{newpxtext}\usepackage{newpxmath}\usepackage{commath}\usepackage{mathtools}")
#plt.rc("font", family="serif", size=14.)
#plt.rc("savefig", dpi=200)
plt.rc("legend", fontsize="medium", fancybox=True, framealpha=0.5)

In [ ]:
def RRK(rkm, dt, f, w0=[1.,0], t_final=1., relaxation=True, 
        rescale_step=True, debug=False, gammatol=0.1, print_gamma=False,
        one_step=False, dissip_factor=1.0):
    """
    Relaxation Runge-Kutta method implementation.
    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time
        relaxation: if True, use relaxation method.  Otherwise, use vanilla RK method.
        rescale_step: if True, new time step is t_n + \gamma dt
        debug: output some additional diagnostics
        gammatol: Fail if abs(1-gamma) exceeds this value
        
    """
    w = np.array(w0)
    t = 0
    # We pre-allocate extra space because if rescale_step==True then
    # we don't know exactly how many steps we will take.
    ww = np.zeros([len(w0),int((t_final-t)/dt*2.5)+10000])
    ww[:,0] = w.copy()
    tt = [t]
    ii = 0
    s = len(rkm)
    b = rkm.b
    y = np.zeros((s,len(w0)))
    max_gammam1 = 0.
    gams = []
    
    while t < t_final:
        if t + dt >= t_final:
            dt = t_final - t # Hit final time exactly
        
        for i in range(s):
            y[i,:] = w.copy()
            for j in range(i):
                y[i,:] += rkm.A[i,j]*dt*f(y[j,:])
                
        F = np.array([f(y[i,:]) for i in range(s)])
        
        if relaxation:
            numer = 2*sum(b[i]*rkm.A[i,j]*np.dot(F[i],F[j]) \
                                for i in range(s) for j in range(s))
            denom = sum(b[i]*b[j]*np.dot(F[i],F[j]) for i in range(s) for j in range(s))
            if denom != 0:
                gam = numer/denom
            else:
                gam = 1.
        else:  # Use standard RK method
            gam = 1.
           
        if print_gamma:
            print(gam)
        
        if np.abs(gam-1.) > gammatol:
            print(gam)
            raise Exception("The time step is probably too large.")
        
        w = w + dissip_factor*gam*dt*sum([b[j]*F[j] for j in range(s)])
        if (t+dt < t_final) and rescale_step:
            t += dissip_factor*gam*dt
        else:
            t += dt
        ii += 1
        tt.append(t)
        ww[:,ii] = w.copy()
        if debug:
            gm1 = np.abs(1.-gam)
            max_gammam1 = max(max_gammam1,gm1)
            gams.append(gam)
            
        if one_step:
            return w, gam
            
    if debug:
        print(max_gammam1)
        return tt, ww[:, :ii+1], np.array(gams)
    else:
        return tt, ww[:,:ii+1]

# Introductory example (Ranocha's problem)

In [ ]:
def f_prob1(u):
    E = u[0]**2 + u[1]**2
    return np.array([-u[1],u[0]])/E

In [ ]:
plt.rc("font", size=20)

In [ ]:
u0 = np.array([1.,0.])
dt = 0.1
t_final = 10.

for i, method in enumerate(short_methods):
    tt, uu = RRK(method, dt, f_prob1, w0=u0, t_final=t_final, 
                                            relaxation=0, debug=False)
    plt.semilogy(tt[1:],uu[0,1:]**2+uu[1,1:]**2-1., lw=3, label=short_method_labels[i]);


plt.xlabel('$t$');
plt.ylabel('$\|u(t)\|^2-\|u(0)\|^2$')
plt.tight_layout()
plt.savefig('./figures/problem1_energy_RK.pdf')

ax = plt.gca()
plt.figure()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=4)
plt.savefig("./figures/problem1_legend.pdf", bbox_inches="tight")

In [ ]:
u0 = np.array([1.,0.])
dt = 0.1

for method in short_methods:
    tt, uu = RRK(method, dt, f_prob1, w0=u0, t_final=10., 
                                            relaxation=1, debug=False)
    plt.plot(tt,uu[0,:]**2+uu[1,:]**2-1.,lw=2.5);


plt.xlabel('$t$');
plt.ylabel('$\|u(t)\|^2-\|u(0)\|^2$')
plt.tight_layout()
plt.savefig('./figures/problem1_energy_RRK.pdf')

In [ ]:
plt.rc("font", size=16)

#names = ['SSP(2,2)', 'SSP(3,3)', 'RK(4,4)', 'BS(8,5)']
dts = np.array([0.9, 0.5, 0.2, 0.1, 0.03, 0.02, 0.011, 0.005])
dts = np.logspace(-2.5,0.,11)
u0 = np.array([1.,0.])
t_final = 10.

fig=plt.figure(figsize=(6,6))
ax = plt.subplot(111)
lines = []

for j, rkm in enumerate(short_methods):
    for relax in (False, True):
        sols = []
        for dt in dts:

            tt, uu = RRK(rkm, dt,f_prob1, w0=u0, t_final=t_final, relaxation=relax, rescale_step=True, gammatol=1.0)
            sols.append(uu[0,-1])

        errors = []
        ref = np.cos(t_final)
        for i in range(len(sols)):
            errors.append(abs(ref-sols[i]))

        if relax:
            plt.loglog(dts, errors,linestyle='dashed',color=colors[j],marker=markers[j],label=None);
        else:
            lines.append(plt.loglog(dts, errors,linestyle='-',color=colors[j],
                                    marker=markers[j],label=short_method_labels[j])[0])
        
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(lines,short_method_labels,loc='center left', bbox_to_anchor=(1, 0.5))

plt.xlabel('$\Delta t$')
plt.ylabel('Error');
plt.ylim([1.e-14, 10]);

plt.plot([1e-2,4e-2],[3.e-4, 4**2*3.e-4],'-ok',alpha=0.5)
plt.plot([1e-2,4e-2],[5.e-6, 4**3*5.e-6],'-ok',alpha=0.5)
plt.plot([1e-2,4e-2],[3.e-10, 4**4*3.e-10],'-ok',alpha=0.5)
plt.plot([1e-2,4.e-2],[5.e-13, 4**5*5.e-13],'-ok',alpha=0.5)
plt.plot([4e-2,16e-2],[5.e-13, 4**6*5.e-13],'-ok',alpha=0.5)

plt.text(1.e-2,3.e-4,'$p=2$',verticalalignment='bottom',horizontalalignment='right')
plt.text(1.e-2,5.e-6,'$p=3$',verticalalignment='top',horizontalalignment='left')
plt.text(3.e-2,6.e-9,'$p=4$',verticalalignment='bottom',horizontalalignment='left')
plt.text(1.e-2,5.e-13,'$p=5$',verticalalignment='bottom',horizontalalignment='right')
plt.text(4.e-2,5.e-13,'$p=6$',verticalalignment='top',horizontalalignment='left')

plt.tight_layout()
plt.savefig('./figures/prob1_convergence.pdf', bbox_inches="tight")

In [ ]:
dt = 0.5
tt, uu = RRK(ssp3, dt, f_prob1, w0=u0, t_final=1000., 
                                            relaxation=0, debug=False)
plt.plot(uu[0,:],uu[1,:],color='#56B4E9')
tt, uu = RRK(ssp3, dt, f_prob1, w0=u0, t_final=1000., 
                                            relaxation=1, debug=False)
plt.plot(uu[0,:],uu[1,:],'-k')
plt.plot(uu[0,0],uu[1,0],'ok')
plt.axis('equal');

In [ ]:
def makeplot(dt):
    plt.figure(figsize=(8,8))
    #dt = 1.5
    unp1g, gam = RRK(ssp2, dt, f_prob1, w0=u0, t_final=1000., 
                                                relaxation=1, debug=False,one_step=True,gammatol=1.0)
    du = (unp1g-u0)/gam
    unp1 = u0+du
    plt.plot([u0[0],unp1g[0]],[u0[1],unp1g[1]],'o-',color='#56B4E9')
    plt.plot([u0[0],unp1[0]],[u0[1],unp1[1]],color='r')

    t = np.linspace(0,2*np.pi,1000)
    plt.plot(np.cos(t),np.sin(t),'-k',lw=0.5)
    plt.plot(np.cos(dt),np.sin(dt),'ok')
    plt.plot(np.cos(gam*dt),np.sin(gam*dt),'ob')
    unorm = np.sqrt(unp1[0]**2+unp1[1]**2)
    plt.plot(unp1[0]/unorm,unp1[1]/unorm,'og')
    plt.axis('equal');
    
makeplot(1.5)

In [ ]:
from ipywidgets import widgets, interact

interact(makeplot,dt=widgets.FloatSlider(min=0.1,max=1.5));

# Stability regions

In [ ]:
plt.rc("font", size=22)

def relaxed_stability_polynomial_gammaval(rkm,gam):
    P, Q = rkm.stability_function()
    
    coeffs = P.coef
    for i in range(len(P)):
        coeffs[i] = gam*coeffs[i]
    pg = np.poly1d(coeffs)
    return pg

def plot_perturbed_stability_regions(rkm,gammas):
    fig=plt.figure(figsize=(12,12))
    q = np.poly1d([1])
    for g in gammas:
        p = relaxed_stability_polynomial_gammaval(rkm,g)
        stability_function.plot_stability_region(p,q,filled=False,color='b',
                                                 fignum=fig.number,alpha=1.6-g**1.4,
                                                 bounds=[-3,0.6,-3.3,3.3])#,
                                                 #plot_axes=False);
    p = relaxed_stability_polynomial_gammaval(rkm,1.)
    stability_function.plot_stability_region(p,q,filled=False,color='k',bounds=[-3,0.6,-3.3,3.3],fignum=fig.number);

In [ ]:
gammas = [0.7,0.8,0.9,1.0,1.1,1.2,1.3]

plot_perturbed_stability_regions(ssp2,gammas)
plt.savefig('./figures/RK2_stability_perturbed.pdf', bbox_inches="tight")

In [ ]:
plot_perturbed_stability_regions(ssp3,gammas)
plt.savefig('./figures/RK3_stability_perturbed.pdf', bbox_inches="tight")

In [ ]:
plot_perturbed_stability_regions(rk4,gammas)
plt.savefig('./figures/RK4_stability_perturbed.pdf', bbox_inches="tight")

# Spectral advection

In [ ]:
def RRK_with_spectrum(rkm, dt, f, w0=[1.,0], t_final=1., relaxation=True, 
                      rescale_step=True, debug=False, gammatol=0.1):
    w = np.array(w0)
    t = 0
    ww = np.zeros([len(w0),int((t_final-t)/dt*1.5)])
    ww[:,0] = w.copy()
    tt = [t]
    ii = 0
    b = rkm.b
    s = len(rkm)
    y = np.zeros((s,len(w0)))
    max_gammam1 = 0.
    gams = []
    uhats = [np.abs(np.fft.fft(w0))]
    while t < t_final:
        if t + dt >= t_final:
            dt = t_final - t
        
        for i in range(s):
            y[i,:] = w.copy()
            for j in range(i):
                y[i,:] += rkm.A[i,j]*dt*f(y[j,:])
                
        F = np.array([f(y[i,:]) for i in range(s)])
        
        if relaxation:
            numer = 2*sum(b[i]*rkm.A[i,j]*np.dot(F[i],F[j]) \
                                for i in range(s) for j in range(s))
            denom = sum(b[i]*b[j]*np.dot(F[i],F[j]) for i in range(s) for j in range(s))
            if denom != 0:
                gam = numer/denom
            else:
                gam = 1.
        else:  # Use standard RK method
            gam = 1.
            
        gm1 = np.abs(1.-gam)
        max_gammam1 = max(max_gammam1,gm1)
        if gm1 > gammatol:
            print(gam)
            raise Exception("The time step is probably too large.")
        
        w = w + gam*dt*sum([b[j]*F[j] for j in range(s)])
        if (t+dt < t_final) and rescale_step:
            t += gam*dt
        else:
            t += dt
        ii += 1
        tt.append(t)
        ww[:,ii] = w.copy()
        gams.append(gam)
        uhats.append(np.abs(np.fft.fft(w)))
    if debug:
        print(max_gammam1)
        return tt, ww[:, :ii+1], np.array(gams), uhats
    return tt, ww[:,:ii+1], uhats

In [ ]:
plt.rc("font", size=20)

m=128
L = 2*np.pi
x = np.arange(-m/2,m/2)*(L/m)
xi = np.fft.fftfreq(m)*m*2*np.pi/L
dx = x[1]-x[0]

def f_advection(u):
    uhat = np.fft.fft(u)
    return -np.real(np.fft.ifft(1j*xi*uhat))

In [ ]:
def plot_rel_amplification(rkname, IC='sech2', relaxation=True, mus=(0.2, 0.5, 0.9, 0.99), filename=None):
    if IC == 'sech2':
        u0 = 1./np.cosh(7.5*(x+1))**2
    elif IC == 'whitenoise':
        theta = np.random.rand(*xi.shape)
        u0hat = np.exp(1j*theta)
        u0 = np.real(np.fft.ifft(u0hat))

    rkm = rk.loadRKM(rkname)
    stabint = rkm.imaginary_stability_interval()
    if stabint == 0: stabint = 1.
    plt.figure(figsize=(6,5))
    for mu in mus:
        dt = mu * stabint * L/m/np.pi
        tt, uu, uhats = RRK_with_spectrum(rkm.__num__(), dt, f_advection, w0=u0, t_final=1., 
                                            relaxation=relaxation, debug=False, gammatol=1.0)
        uhatdiff = np.ma.array(uhats[-1]-uhats[0])
        uhatreldiff = uhatdiff/uhats[0]
        plt.plot(np.sort(xi)[m//2:],uhatreldiff[np.argsort(xi)][m//2:],'-',lw=3,label='$\mu = ' + str(mu) + '$')
    plt.xlabel('Wavenumber')
    plt.ylabel('Relative amplification')
    #plt.legend(['$\mu = ' + str(mu) + '$' for mu in np.array(mus)],loc=3);
    plt.xlim(0,m/2-1)
    plt.ylim(-1.2,0.8)
    plt.tight_layout()
    if filename:
        plt.savefig(filename)
    
    ax = plt.gca()
    plt.figure()
    handles, labels = ax.get_legend_handles_labels()
    plt.figlegend(handles, labels, loc="center", ncol=5)
    plt.savefig("./figures/waveeq_amp_legend.pdf", bbox_inches="tight");

In [ ]:
plot_rel_amplification('RK44','whitenoise',relaxation=False,
                       filename='./figures/waveeq_RK44_standard.pdf')


In [ ]:
plot_rel_amplification('RK44','whitenoise',relaxation=True,
                       filename='./figures/waveeq_RK44_whitenoise_relaxation.pdf')

In [ ]:
plot_rel_amplification('RK44','sech2',relaxation=True,mus=(0.2, 0.5, 0.9, 0.99, 1.02),
                       filename='./figures/waveeq_RK44_sech2_relaxation.pdf')

In [ ]:
def compare_solutions(rkname='RK44', relaxation=False, mus= (0.5, 0.9), t_final=100*np.pi, plot_exact=True, lw=3):

    rkm = rk.loadRKM(rkname)
    u0 = 1./np.cosh(7.5*(x+1))**2
    if plot_exact: plt.plot(x,u0,'-k',alpha=0.5)

    stabint = rkm.imaginary_stability_interval()
    if stabint == 0: stabint = 1.

    for mu in mus:
        dt = mu * stabint * L/m/np.pi

        tt, uu, gams = RRK(rkm.__num__(), dt, f_advection, w0=u0, t_final=t_final, 
                                                relaxation=relaxation, debug=True, gammatol=1.0)
        plt.plot(x,uu[:,-1],lw=lw)

    plt.xlim(-np.pi,np.pi);

In [ ]:
t_final = 400*np.pi
plt.figure(figsize=(6,4))
compare_solutions('RK44',t_final=t_final,mus=(0.99,))
compare_solutions('RK44',t_final=t_final,mus=(0.99,),relaxation=True,plot_exact=False)
plt.legend(['Exact','RK4','RRK4']);
plt.xlabel('$x$'); plt.ylabel('$u$');
plt.savefig('./figures/waveeq_RK4_comparison.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(6,4))
compare_solutions('RK44',t_final=t_final,mus=(1.015,),relaxation=True)
compare_solutions('RK44',t_final=t_final,mus=(1.016,),relaxation=True,plot_exact=False,lw=1.5)
plt.legend(['Exact','$\mu=1.015$','$\mu=1.016$']);
plt.xlabel('$x$'); plt.ylabel('$u$');
plt.savefig('./figures/waveeq_RRK4_threshold.pdf', bbox_inches="tight")

# Sun-Shu problem

In [ ]:
A = -np.array([[1,2,2],[0,1,2.],[0,0,1.]])
dt = 0.5

def f_SS(w):
    return np.dot(A,w)

P4 = np.eye(3) + dt*A + 0.5*dt**2*np.dot(A,A) + 1./6 * dt**3 * np.linalg.matrix_power(A,3) \
     + 1./24*dt**4*np.linalg.matrix_power(A,4)
U, S, V = np.linalg.svd(P4)
np.dot(P4,V.T)-np.dot(U,np.diag(S))
w0 = V.T[:,0]
print(S[0])
print(w0)

In [ ]:
#RK4x2
dts = np.linspace(1.1,1.6)

for dt in dts:

    P4 = np.eye(3) + dt*A + 0.5*dt**2*np.dot(A,A) + 1./6 * dt**3 * np.linalg.matrix_power(A,3) \
         + 1./24*dt**4*np.linalg.matrix_power(A,4) + 1./128*dt**5*np.linalg.matrix_power(A,5) \
         + 1./9216*dt**6*np.linalg.matrix_power(A,6) \
         + 1./9216*dt**7*np.linalg.matrix_power(A,7) \
         + 1./147456*dt**8*np.linalg.matrix_power(A,8)
    U, S, V = np.linalg.svd(P4)
#    print(dt, S[0])

#w0 = V.T[:,0]
#print(w0)
#np.linalg.norm(np.dot(P4,w0),2)

In [ ]:
plt.rc("font", size=16)
method = rk4
plt.figure(figsize=(6,4))
lw = 2.5

tt, ww = RRK(method,0.7,f_SS,w0=w0,t_final=1,relaxation=False)
E = (ww**2).sum(0)
plt.plot(tt,E,color=colors[2],marker=markers[0],lw=lw);

tt, ww = RRK(method,0.5,f_SS,w0=w0,t_final=1,relaxation=False)
E = (ww**2).sum(0)
plt.plot(tt,E,color=colors[2],marker=markers[1],lw=lw);
print(E[0]-E[1])

tt, wwes = RRK(method,0.7,f_SS,w0=w0,t_final=1,relaxation=True, gammatol=2.0)
Ees = (wwes**2).sum(0)
plt.plot(tt,Ees,color=colors[3],marker=markers[2],lw=lw);

tt, wwes = RRK(method,0.5,f_SS,w0=w0,t_final=1,relaxation=True, gammatol=2.0)
Ees = (wwes**2).sum(0)
plt.plot(tt,Ees,color=colors[3],marker=markers[3],lw=lw);

tt, ww = RRK(rk4,0.01,f_SS,w0=w0,t_final=1,relaxation=False)
E = (ww**2).sum(0)
plt.plot(tt,E,'-k');

plt.ylabel('$\|u\|^2$'); plt.xlabel('$t$')

ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(['RK4, $\Delta t=0.7$', 'RK4, $\Delta t=0.5$', 'RRK4, $\Delta t=0.7$', 'RRK4, $\Delta t=0.5$', 'exact'],
          loc='center left', bbox_to_anchor=(1, 0.5))

#plt.legend(['RK4, $\Delta t=0.7$', 'RK4, $\Delta t=0.5$', 'RRK4, $\Delta t=0.7$', 'RRK4, $\Delta t=0.5$', 'exact'])
plt.ylim([0.94,1.02])
plt.savefig('./figures/sunshu_rk4.pdf', bbox_inches="tight")

# Burgers' equation

In [ ]:
plt.rc("font", size=16)

def burgers_energy(rkms, tfinal=0.02, relaxation=True, eps=0., cflnum=0.1, plotabs=True, plot_exact=False):
    x = np.linspace(-1,1,50)
    dx = x[1]-x[0]
    u0 = np.exp(-30*x**2)

    def f(u):
        Fiph = np.zeros_like(u)
        fluxdiff = np.zeros_like(u)
        Fiph[:-1] = (u[1:]**2 + u[1:]*u[:-1] + u[:-1]**2) - eps*(u[1:]-u[:-1])  # Entropy dissipative
        fluxdiff[1:] = -(Fiph[1:]-Fiph[:-1])/dx
        return fluxdiff
            
    dt = cflnum*dx
    
    for rkm in rkms:
        if relaxation:
            tt, uu = RRK(rkm, dt,f,w0=u0,t_final=tfinal,relaxation=True,gammatol=1.)
        else:
            tt, uu = RRK(rkm, dt,f,w0=u0,t_final=tfinal,relaxation=False)

        E = [sum(uu[:,i]**2) for i in range(uu.shape[1])]
        if plotabs:
            plt.semilogy(tt[1:],np.abs(E[1:]-E[0]),lw=3);
            plt.ylabel('$|\|u^{n}\|^2 - \|u^{0}\|^2|$'); plt.xlabel('$t$')
        else:
            plt.plot(tt[1:],(E[1:]-E[0])/E[0],lw=3);
            plt.ylabel('$(\|u^{n}\|^2 - \|u^{0}\|^2)/\|u^{0}\|^2$'); plt.xlabel('$t$')

    if plot_exact:
        dt = 0.01*cflnum*dx
        tt, uu = RRK(bs5, dt, f, w0=u0, t_final=tfinal, relaxation=False)
        E = [sum(uu[:,i]**2) for i in range(uu.shape[1])]
        if plotabs:
            plt.semilogy(tt[1:],np.abs(E[1:]-E[0]),'-k',lw=1);
            plt.ylabel('$|\|u^{n}\|^2 - \|u^{0}\|^2|$'); plt.xlabel('$t$')
        else:
            plt.plot(tt[1:],(E[1:]-E[0])/E[0],'-k',lw=1);
            plt.ylabel('$(\|u^{n}\|^2 - \|u^{0}\|^2)/\|u^{0}\|^2$'); plt.xlabel('$t$')
        plt.legend(short_method_labels[1:]+['Reference'])    
    else:
        plt.legend(short_method_labels[1:])

In [ ]:
burgers_energy(short_methods[1:],tfinal=0.2,relaxation=False,cflnum=0.3, eps=0.)
plt.savefig('./figures/burgers_energy_rk.pdf', bbox_inches="tight")

In [ ]:
burgers_energy(short_methods[1:],tfinal=0.2,relaxation=True,cflnum=0.3,plotabs=False)
plt.savefig('./figures/burgers_energy_rrk.pdf', bbox_inches="tight")

In [ ]:
x = np.linspace(-1,1,6400,endpoint=False)
dx = x[1]-x[0]
u0 = np.exp(-30*x**2)
t_final = 0.03

def f(u):
    Fiph = np.zeros_like(u)
    fluxdiff = np.zeros_like(u)
    #Fiph[1:] = u[1:]
    #Fiph[:-1] = (u[1:]**2 + u[:-1]**2)/4.
    Fiph[:-1] = (u[1:]**2 + u[1:]*u[:-1] + u[:-1]**2)  # Entropy conservative
    fluxdiff[1:] = -(Fiph[1:]-Fiph[:-1])/dx
    return fluxdiff

dt = 0.1*dx
tt, uu = RRK(rk4, dt,f,w0=u0,t_final=t_final,relaxation=False)
ref = uu[:,-1]
xref = x.copy()
u0ref = u0.copy()
plt.plot(xref,ref)

In [ ]:
N = 50
mu = 1.5
t_final = 0.03
base = 1./5
cfls = base*0.5**np.arange(7)
#cfls = (0.2, 0.1, 0.05, 0.025, 0.0125)
rkms = [ssp3, rk4, bs5]
lines = []

plt.figure(figsize=(6,8))
for j, rkm in enumerate(short_methods):
    for relax in (False, True):
        sols = []
        xs = []
        for cfl in cfls:
            x = np.linspace(-1,1,N,endpoint=False)
            dx = x[1]-x[0]
            u0 = np.exp(-30*x**2)

            def f(u):
                Fiph = np.zeros_like(u)
                fluxdiff = np.zeros_like(u)
                Fiph[:-1] = (u[1:]**2 + u[1:]*u[:-1] + u[:-1]**2)
                fluxdiff[1:] = -(Fiph[1:]-Fiph[:-1])/dx
                return fluxdiff

            dt = cfl*dx
            tt, uu = RRK(rkm, dt,f,w0=u0,t_final=t_final,relaxation=relax,rescale_step=True,gammatol=1.0)
            sols.append(uu[:,-1])
            xs.append(x)

        errors = []
        ref = sols[-1]
        for i in range(len(sols)):
            errors.append(sum(abs(ref-sols[i])))

        if relax:
            plt.loglog(cfls[:-1], errors[:-1],linestyle='dashed',color=colors[j],marker=markers[j],label=None);
        else:
            lines.append(plt.loglog(cfls[:-1], errors[:-1],linestyle='-',color=colors[j],
                                    marker=markers[j],label=short_method_labels[j])[0])
            

plt.plot([1e-2,4*1e-2],[8.e-4, 4**2*8.e-4],'-ok',alpha=0.5)
plt.plot([1e-2,4*1e-2],[1.e-5, 4**3*1.e-5],'-ok',alpha=0.5)
plt.plot([1e-2,4*1e-2],[3.e-8, 4**4*3.e-8],'-ok',alpha=0.5)
plt.plot([1e-2,4*1.e-2],[5.e-12, 4**5*5.e-12],'-ok',alpha=0.5)

plt.text(1.e-2,8.e-4,'$p=2$',verticalalignment='bottom',horizontalalignment='right')
plt.text(1.e-2,2.e-4,'$p=3$',verticalalignment='top',horizontalalignment='left')
plt.text(3.e-2,8.e-7,'$p=4$',verticalalignment='bottom',horizontalalignment='left')
plt.text(1.e-2,5.e-12,'$p=5$',verticalalignment='bottom',horizontalalignment='right')

plt.legend(lines,short_method_labels)
plt.xlabel('$\Delta t$')
plt.ylabel('Error');
plt.ylim([1.e-14, 1])
plt.savefig('./figures/burgers_convergence.pdf', bbox_inches="tight")

In [ ]:
N = 50
mu = 1.5
base = 1./5
cfls = base*0.5**np.arange(7)
#cfls = (0.2, 0.1, 0.05, 0.025, 0.0125)
rkms = [ssp3, rk4, bs5]

plt.figure(figsize=(6,8))
for j, rkm in enumerate(short_methods):
    for relax in (False, True):
        sols = []
        xs = []
        for cfl in cfls:
            x = np.linspace(-1,1,N,endpoint=False)
            dx = x[1]-x[0]
            u0 = np.exp(-30*x**2)

            def f(u):
                Fiph = np.zeros_like(u)
                fluxdiff = np.zeros_like(u)
                Fiph[:-1] = (u[1:]**2 + u[1:]*u[:-1] + u[:-1]**2)
                fluxdiff[1:] = -(Fiph[1:]-Fiph[:-1])/dx
                return fluxdiff

            dt = cfl*dx
            tt, uu = RRK(rkm, dt,f,w0=u0,t_final=t_final,relaxation=relax,rescale_step=False)
            sols.append(uu[:,-1])
            xs.append(x)

        errors = []
        ref = sols[-1]
        for i in range(len(sols)):
            errors.append(sum(abs(ref-sols[i])))

        if relax:
            plt.loglog(cfls[:-1], errors[:-1],linestyle='dashed',color=colors[j],marker=markers[j],label=None);
        else:
            lines.append(plt.loglog(cfls[:-1], errors[:-1],linestyle='-',color=colors[j],
                                    marker=markers[j],label=short_method_labels[j])[0])
            
        
plt.legend(lines,short_method_labels)
plt.xlabel('$\Delta t$')
plt.ylabel('Error');
plt.ylim([1.e-14, 1])
plt.savefig('./figures/burgers_convergence_fixedstep.pdf', bbox_inches="tight")

In [ ]:
burgers_energy(short_methods[1:],tfinal=0.2,relaxation=False,cflnum=0.3,eps=0.01,plotabs=False,plot_exact=True)
plt.savefig('./figures/burgersdiss_energy_RK.pdf', bbox_inches="tight")

In [ ]:
burgers_energy(short_methods[1:],tfinal=0.2,relaxation=True,cflnum=0.3,eps=0.01,plotabs=False,plot_exact=True)
plt.savefig('./figures/burgersdiss_energy_RRK.pdf', bbox_inches="tight")

In [ ]:
N = 50
mu = 1.5
t_final = 0.03
base = 1./5
cfls = base*0.5**np.arange(7)
rkms = [ssp3, rk4, bs5]
eps=0.01

plt.figure(figsize=(6,8))
for j, rkm in enumerate(short_methods):
    for relax in (False, True):
        sols = []
        xs = []
        for cfl in cfls:
            x = np.linspace(-1,1,N,endpoint=False)
            dx = x[1]-x[0]
            u0 = np.exp(-30*x**2)

            def f(u):
                Fiph = np.zeros_like(u)
                fluxdiff = np.zeros_like(u)
                Fiph[:-1] = (u[1:]**2 + u[1:]*u[:-1] + u[:-1]**2)  - eps*(u[1:]-u[:-1])
                fluxdiff[1:] = -(Fiph[1:]-Fiph[:-1])/dx
                return fluxdiff

            dt = cfl*dx
            tt, uu = RRK(rkm, dt,f,w0=u0,t_final=t_final,relaxation=relax,rescale_step=True,gammatol=1.0)
            sols.append(uu[:,-1])
            xs.append(x)

        errors = []
        ref = sols[-1]
        for i in range(len(sols)):
            errors.append(sum(abs(ref-sols[i])))

        if relax:
            plt.loglog(cfls[:-1], errors[:-1],linestyle='dashed',color=colors[j],marker=markers[j],label=None);
        else:
            lines.append(plt.loglog(cfls[:-1], errors[:-1],linestyle='-',color=colors[j],
                                    marker=markers[j],label=short_method_labels[j])[0])
            
plt.plot([1e-2,4*1e-2],[1.e-3, 4**2*1.e-3],'-ok',alpha=0.5)
plt.plot([1e-2,4*1e-2],[9.e-6, 4**3*9.e-6],'-ok',alpha=0.5)
plt.plot([1e-2,4*1e-2],[3.e-8, 4**4*3.e-8],'-ok',alpha=0.5)
plt.plot([1e-2,4*1.e-2],[2.e-12, 4**5*2.e-12],'-ok',alpha=0.5)

plt.text(1.e-2,1.e-3,'$p=2$',verticalalignment='bottom',horizontalalignment='right')
plt.text(1.e-2,9.e-5,'$p=3$',verticalalignment='top',horizontalalignment='left')
plt.text(3.e-2,8.e-7,'$p=4$',verticalalignment='bottom',horizontalalignment='left')
plt.text(1.e-2,2.e-12,'$p=5$',verticalalignment='bottom',horizontalalignment='right')
            
plt.legend(lines,short_method_labels)
plt.xlabel('$\Delta t$')
plt.ylabel('Error');
plt.ylim([1.e-14, 1])
plt.savefig('./figures/burgersdiss_convergence.pdf', bbox_inches="tight")

In [ ]:
N = 50
mu = 1.5
base = 1./5
cfls = base*0.5**np.arange(7)
rkms = [ssp3, rk4, bs5]
eps=0.01

plt.figure(figsize=(6,8))
for j, rkm in enumerate(short_methods):
    for relax in (False, True):
        sols = []
        xs = []
        for cfl in cfls:
            x = np.linspace(-1,1,N,endpoint=False)
            dx = x[1]-x[0]
            u0 = np.exp(-30*x**2)

            def f(u):
                Fiph = np.zeros_like(u)
                fluxdiff = np.zeros_like(u)
                Fiph[:-1] = (u[1:]**2 + u[1:]*u[:-1] + u[:-1]**2)  - eps*(u[1:]-u[:-1])
                fluxdiff[1:] = -(Fiph[1:]-Fiph[:-1])/dx
                return fluxdiff

            dt = cfl*dx
            tt, uu = RRK(rkm, dt,f,w0=u0,t_final=t_final,relaxation=relax,rescale_step=False)
            sols.append(uu[:,-1])
            xs.append(x)

        errors = []
        ref = sols[-1]
        for i in range(len(sols)):
            errors.append(sum(abs(ref-sols[i])))

        if relax:
            plt.loglog(cfls[:-1], errors[:-1],linestyle='dashed',color=colors[j],marker=markers[j],label=None);
        else:
            lines.append(plt.loglog(cfls[:-1], errors[:-1],linestyle='-',color=colors[j],
                                    marker=markers[j],label=short_method_labels[j])[0])
            
        
plt.legend(lines,short_method_labels)
plt.xlabel('$\Delta t$')
plt.ylabel('Error');
plt.ylim([1.e-14, 1])
plt.savefig('./figures/burgersdiss_convergence_fixedstep.pdf', bbox_inches="tight")

In [ ]:
x = np.linspace(-1,1,50)
dx = x[1]-x[0]
u0 = (x>-0.5)*(x<0.5)
t_final = 0.1
eps = 0.1

def f(u):
    Fiph = np.zeros_like(u)
    fluxdiff = np.zeros_like(u)
    Fiph[:-1] = (u[1:]**2 + u[1:]*u[:-1] + u[:-1]**2) - eps*(u[1:]-u[:-1])
    fluxdiff[1:] = -(Fiph[1:]-Fiph[:-1])/dx
    return fluxdiff

dt = 0.3*dx
tt, uu = RRK(rk4, dt,f,w0=u0,t_final=t_final,relaxation=False)
ref = uu[:,-1]
xref = x.copy()
u0ref = u0.copy()
plt.plot(xref,ref)
tt, uu = RRK(rk4, dt,f,w0=u0,t_final=t_final,relaxation=True)
plt.plot(xref,uu[:,-1])

In [ ]:
dt = 0.3*dx
u1, gam = RRK(rk4, dt, f, w0=u0, t_final=t_final, relaxation=False, one_step=True)
du = u1-u0
gammas = np.linspace(-0.1,1.2)
E0= np.linalg.norm(u0,2)**2
Es = [np.linalg.norm(u0+gamma*du,2)**2 for gamma in gammas]
plt.plot(gammas, Es-E0,lw=3)
plt.plot([-0.2,1.2],[0,0],'--k')
plt.xlabel('$\gamma$')
plt.ylabel('Change in energy')
plt.savefig('gamma_variation.pdf',bbox_inches='tight')

In [ ]:
x = np.linspace(-1,1,50)
dx = x[1]-x[0]
u0 = (x>-0.5)*(x<0.5)
t_final = 0.1
eps = 1.

def f(u):
    Fiph = np.zeros_like(u)
    fluxdiff = np.zeros_like(u)
    Fiph[:-1] = (u[1:]**2 + u[1:]*u[:-1] + u[:-1]**2) - eps*(u[1:]-u[:-1])
    fluxdiff[1:] = -(Fiph[1:]-Fiph[:-1])/dx
    return fluxdiff

dt = 0.33*dx
tt, uu = RRK(rk4, dt,f,w0=u0,t_final=t_final,relaxation=False)
ref = uu[:,-1]
xref = x.copy()
u0ref = u0.copy()
plt.plot(xref,ref,lw=3)
tt, uu = RRK(rk4, dt,f,w0=u0,t_final=t_final,relaxation=True,dissip_factor=0.5)
plt.plot(xref,uu[:,-1],'--o',lw=3)
plt.legend(['$\hat{\gamma}=\gamma$','$\hat{\gamma}=0.5\gamma$'])
plt.savefig('gamma_dissipation.pdf')